# 🏗️ Arquitectura de Autenticación Empresarial – Microsoft Foundry (Secure Secret Integration)

> **Propiedad Intelectual © 2026 – Eduardo Osorio Venegas**  
> Todos los derechos reservados.

---

## 📌 Descripción del Ejercicio

Implementación de consumo seguro de un modelo desplegado en **Microsoft Foundry** mediante autenticación con **Service Principal (Microsoft Entra ID)**, utilizando el flujo **OAuth 2.0 Client Credentials**, con integración segura de credenciales almacenadas en **Azure Key Vault**.

A diferencia de implementaciones tradicionales basadas en archivos `.env`, este proyecto centraliza la gestión de secretos en un **Key Vault existente**, eliminando la exposición de credenciales en código o configuraciones locales.

---

## ✅ Validaciones del Ejercicio

El ejercicio valida:

- Emisión de token JWT vía **Microsoft Entra ID**.
- Obtención segura de credenciales desde **Azure Key Vault**.
- Autorización contra endpoint de **Microsoft Foundry**.
- Consumo del modelo mediante **REST API**.
- Aplicación de buenas prácticas de seguridad en entornos **Azure**:
  - Principio de menor privilegio.
  - Eliminación de secretos en texto plano.
  - Centralización y rotación segura de credenciales.
  - Integración con RBAC y Managed Identities (cuando aplica).

# Instalar librerias

In [1]:
#! pip install azure-cli

In [2]:
#%pip install azure-identity azure-keyvault-secrets

# Leer secreto desde Key Vault

In [3]:
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient
import os
from dotenv import load_dotenv

In [4]:
load_dotenv()

print("KEY_VAULT_URL:", os.getenv("KEY_VAULT_URL"))

KEY_VAULT_URL: 


# Autenticación automática

az login  
↓  
Azure CLI guarda el token de autenticación  
↓  
DefaultAzureCredential detecta automáticamente el token  
↓  
Se genera un bearer token para la solicitud  
↓  
Azure OpenAI valida tu identidad  

In [16]:
# Autenticación automática
credential = DefaultAzureCredential()

# Cliente para consumir secretos desde Key Vault

In [21]:
client = SecretClient(vault_url=os.getenv("KEY_VAULT_URL"), credential=credential)

# Obtener secretos
* Diferencia importante entre roles al momento de leer tu secreto desde Key Vault

| Rol                     | ¿Puede leer valor? |
| ----------------------- | ------------------ |
| Key Vault Administrator | ✅ Sí               |
| Key Vault Secrets User  | ✅ Sí               |
| Key Vault Reader        | ❌ No               |
| Owner                   | ✅ Sí               |


In [22]:
tenant_id = client.get_secret("APPFOUNDRYEO-AZURE-TENANT-ID").value
client_id = client.get_secret("APPFOUNDRYEO-AZURE-CLIENT-ID").value
client_secret = client.get_secret("APPFOUNDRYEO-AZURE-CLIENT-SECRET").value
foundry_endpoint = client.get_secret("FOUNDRY-PROJECT-ENDPOINT").value

# Validar conexion proyecto foundry

In [24]:
from azure.ai.projects import AIProjectClient

project_client = AIProjectClient(
    endpoint= client.get_secret("FOUNDRY-PROJECT-ENDPOINT").value,
    credential=credential
)

print("Conexión al AI Project exitosa")

Conexión al AI Project exitosa


# Listar los modelos disponible en Microsoft Foundry del proyecto

In [25]:
deployments = project_client.deployments.list()

for d in deployments:
    print(d.name)

gpt-4.1


# Utilizar el modelo

In [27]:
openai_client = project_client.get_openai_client()

response = openai_client.responses.create(
    model="gpt-4.1",
    input="Que sabes de microsoft foundry?"
)

print(response.output_text)

Microsoft Foundry fue un programa creado por Microsoft, principalmente en México, enfocado en la formación y aceleración de estudiantes universitarios y recién egresados en áreas relacionadas con tecnología y desarrollo de software. El objetivo era acercar al talento joven a la industria de TI, ofreciéndoles una experiencia de trabajo similar a la de un entorno profesional real dentro de Microsoft.

Te comparto algunos puntos clave sobre Microsoft Foundry:

- **Inicio:** El programa inició alrededor de 2012 en México, destacándose primero en la oficina de Microsoft en la Ciudad de México.
- **Formato:** Era una especie de internship (pasantía) o residencia para estudiantes y recién graduados, generalmente de carreras de ingeniería, ciencias computacionales o afines.
- **Duración:** El programa solía durar entre 12 y 16 semanas (trimestral) y era tipo “intensivo”.
- **Enfoque:** Los participantes trabajaban en equipos multidisciplinarios para resolver problemas reales o desarrollar prot